# Tarea 1

Implementar la clase `OisCashflow`.

Integrantes: Gregorio Salaberry  y 
             Rocío Monsalve

## Configuración Inicial

In [1]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum

In [2]:
from finrisk import QC_Financial_3 as Qcf
from IPython.display import Image
from IPython.core.display import HTML
import pandas as pd
from datetime import date

# Modificado por AD
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux
# Fin modificación

from scipy.optimize import root_scalar
from functools import partial

In [3]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [4]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

In [5]:
calendario_1=get_cal

print(calendario_1)

<function get_cal at 0x7f1380516158>


## `Qcf.time_series`

In [6]:
ts = Qcf.time_series()

fecha1 = Qcf.QCDate(13, 1, 1969)
ts[fecha1] = 19690113

fecha2 = Qcf.QCDate(14, 1, 1969)
ts[fecha2] = 19690114

print(f'ts[fecha1]: {ts[fecha1]}\n')

dl = Qcf.time_series_dates(ts)
for d in dl:
    print(d)
print()
    
vl = Qcf.time_series_values(ts)
for v in vl:
    print(v)

ts[fecha1]: 19690113.0

13-1-1969
14-1-1969

19690113.0
19690114.0


## Clase `OisCashflow`

La clase `OisCashflow` es una [`dataclass`](https://realpython.com/python-data-classes/). La tarea consiste en:

- Implementar los 3 métodos de la clase que están con `pass`.
- Implementar la función `present_value`.
- Implementar la función `set_expected_rate`.

In [7]:
#Importar datos para realizar la tarea 
sofr_excel = pd.ExcelFile('../data/SOFR-10012019-10292020.xls')
#repo_rate = pd.read_excel(sofr_excel, 'RepoRate')
data = pd.read_excel(sofr_excel, 'nice format')
#data = pd.read_excel('SOFR-10012019-10292020.xls','nice format')

data.head(5)

#datos['DATE'] = datos['DATE'].dt.date
type(data)
type(data['DATE'])

data.tail(10)
len(data)

data["DATE"] = pd.to_datetime(data["DATE"])

data['DATE'] = data['DATE'].dt.date
data['RATE\n(PERCENT)'] = data['RATE\n(PERCENT)']

# Se visualiza el resultado
data.style.format({'SOFR': '{0:.2%}'})

for i in range(len(data["DATE"])):
     data.iloc[i,0] = Qcf.QCDate(data.iloc[i,0].day, data.iloc[i,0].month,data.iloc[i,0].year)

        
# Se visualiza el resultado
data.style.format({'SOFR': '{0:.2%}'})


,DATE,BENCHMARK NAME,RATE (PERCENT)
0,28-10-2020,SOFR,0.080000
1,27-10-2020,SOFR,0.090000
2,26-10-2020,SOFR,0.090000
3,23-10-2020,SOFR,0.080000
4,22-10-2020,SOFR,0.070000
5,21-10-2020,SOFR,0.070000
6,20-10-2020,SOFR,0.080000
7,19-10-2020,SOFR,0.090000
8,16-10-2020,SOFR,0.090000
9,15-10-2020,SOFR,0.100000


In [8]:
data.tail(5) 

,DATE,BENCHMARK NAME,RATE\n(PERCENT)
265,7-10-2019,SOFR,1.83
266,4-10-2019,SOFR,1.82
267,3-10-2019,SOFR,1.84
268,2-10-2019,SOFR,1.85
269,1-10-2019,SOFR,1.88


In [9]:
valor_tasa = data['RATE\n(PERCENT)']
fixings = Qcf.time_series()
x = len(data['DATE'])

for i in range(x):
    fixings[data.iloc[i,0]] = data.iloc[i,2]/100
    
fecha_prueba = Qcf.QCDate(28, 10, 2020)
fixings[fecha_prueba]


0.0008

In [10]:
#Entender como funciona Qcf.time_series_dates
dates= Qcf.time_series_dates(fixings)
x=dates[0]
print(x)

1-10-2019


In [11]:
#Para utiilizar las fechas que estan en la base de datos entregada
dates_basedatos= Qcf.time_series_dates(fixings)
value_basedatos = Qcf.time_series_values(fixings)

#Crear dataframe para entender los datos que utilizaremos para hacer las funciones
data_frame=pd.DataFrame(dates_basedatos,value_basedatos)

print(data_frame)

                 0
0.0188   1-10-2019
0.0185   2-10-2019
0.0184   3-10-2019
0.0182   4-10-2019
0.0183   7-10-2019
...            ...
0.0007  22-10-2020
0.0008  23-10-2020
0.0009  26-10-2020
0.0009  27-10-2020
0.0008  28-10-2020

[270 rows x 1 columns]


In [12]:
@dataclass # syntactic sugar
class OisCashflow:
    start_date: Qcf.QCDate
    end_date: Qcf.QCDate
    settlement_date: Qcf.QCDate
    notional: float
    currency: Qcf.QCCurrency
    amortization: float
    amort_is_cashflow: bool
    interest_rate: Qcf.QCInterestRate
    on_index: Qcf.InterestRateIndex
    spread: float
    gearing: float
        
    def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        
        """
        Calcula la tasa equivalente desde `self.start_date` a `accrual_date`. La tasa equivalente
        se calcula como:
        accrual_date es el end date de mi ultimo dato
   
        Lo del calendario, es no pedir una fecha que no tendra un valor
       
        (P - 1) * 360 / (accrual_date - self.start_date)
        
        donde P es el producto de los factores de capitalización de todas las tasas overnight
        entre `start_date` y `accrual_date`. Los valores de esas tasas deben estar almacenados
        en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        
        accrual_date --> devengo
        Tasa lineal actual 360. 
        """
        # Número de días entre la fecha inicial y la fecha final
        dias = self.start_date.day_diff(accrual_date)
        
        dates_basedatos= Qcf.time_series_dates(fixings)
        value_basedatos = Qcf.time_series_values(fixings)
            
        P = 1
        
        for i in range(0, (len(data) -1)):
            start_date= dates_basedatos[i]
            final_date = dates_basedatos[i+1]
            
            if (dates_basedatos[i] != accrual_date):
            
                qc_tasa = Qcf.QCInterestRate(value_basedatos[i], Qcf.QCAct360(), Qcf.QCLinearWf())
            
                wf = qc_tasa.wf(start_date, final_date)
            
                P = P*wf
            
            else:  
                P = P
                
                break #para terminar con el for
                
        if (dias==0):   #restrccion cuando accrual_date es igual al start_date y no se indefine el resultado
            tasa_eq=0  
            
        else:
            # Tasa equivalente
            tasa_eq = (P - 1) * 360.0 / dias
        
        return tasa_eq
        
    
    def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        """
        Calcula el interés (plata) devengado desde `self.start_date` a `accrual_date` utilizando la tasa equivalente
        que se calcula con el método anterior.
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        # Número de días entre la fecha inicial y la fecha final
        dias = self.start_date.day_diff(accrual_date)
        
        #Se usa el resultado de la función anterior
        tasa_eq = self.get_accrued_rate(accrual_date, fixings)
        
        interes_flot_ois = self.notional * tasa_eq * dias / 360
        
 
        return interes_flot_ois
    
    
    def amount(self, fixings: Qcf.time_series) -> float:
        """
        Calcula el flujo total al vencimiento (amortización más intereses devengados hasta self.end_date).
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        #Se usa el resultado de la función anterior
        intereses_devengados= self.get_accrued_interest(self.end_date, fixings)
        
        flujo_vencimiento= intereses_devengados + self.amortization
        
        
        return flujo_vencimiento
    

Construcción de un objeto `Qcf.InterestRateIndex`.

In [13]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)

In [14]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),   #start_date
    Qcf.QCDate(28, 10, 2020),   #end_date
    Qcf.QCDate(28, 10, 2020),   #settlement_date
    10000000,    #notional
    Qcf.QCUSD(), #currency
    10000000,     #amortization
    True,   #amort_is_cashflow 
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()), #interest_rate
    sofr,  #on_index
    0,     #spread
    1      #gearing
)

Construcción de una instancia de `OisCashflow`.

## Clase `Prueba de class OisCashflow`

In [15]:
#Prueba de Qcf.QCDate
#Se eligen una fecha para probar la construcción realizada
x=Qcf.QCDate(10, 10, 2019)

funcion1=ois.get_accrued_rate(x,fixings)
print(funcion1)

0.018403455373592337


In [16]:
funcion2=ois.get_accrued_interest(x,fixings)
print(funcion2)    

4600.863843398085


In [17]:
funcion3=ois.amount(fixings)
print(funcion3)

10078517.177790914


In [18]:
try:
    ois.get_accrued_rate(x,fixings)
except Exception as e:
    print('Cambiar algo porque esta mal')

try:
    ois.get_accrued_interest(x,fixings)
except Exception as e:
    print('Cambiar algo porque esta mal')
    
try:
    ois.amount(fixings)
except Exception as e:
    print('Cambiar algo porque esta mal')
    
#Como no imprime nada, la función esta bien

## Funciones

In [19]:
def get_curve_from_dataframe(yf: Qcf.QCYearFraction, wf: Qcf.QCWealthFactor,
                             df_curva: pd.DataFrame) -> Qcf.ZeroCouponCurve:
    """
    Retorna un objeto Qcf.ZeroCouponCurve. Esta función requiere que `df_curva` tenga una columna
    de nombre 'plazo' y una columna de nombre 'tasa'. Se usa interpolación lineal en la curva que
    se retorna.
    """
    plazos = Qcf.long_vec()
    tasas = Qcf.double_vec()
    for row in df_curva.itertuples():
        plazos.append(row.plazo)
        tasas.append(row.tasa)
    curva = Qcf.QCCurve(plazos, tasas)
    curva = Qcf.QCLinearInterpolator(curva)
    tipo_tasa = Qcf.QCInterestRate(0.0, yf, wf)
    curva = Qcf.ZeroCouponCurve(curva, tipo_tasa)
    return curva

In [20]:
### Variables Globales
frmt = {
    'tasa': '{:.6%}',
    'df': '{:.6%}',
    'valor_tasa': '{:.4%}',
    'spread': '{:.4%}',
    'nominal': '{:,.2f}',
    'interes': '{:,.2f}',
    'amortizacion': '{:,.2f}',
    'flujo': '{:,.2f}',
}

In [21]:
#Importar datos para realizar la tarea 
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')
#df_curva = pd.read_excel('20201012_built_sofr_zero.xlsx')

df_curva.head().style.format(frmt)

,plazo,tasa,df
0,1,0.081111%,99.999778%
1,7,0.084051%,99.998388%
2,14,0.077967%,99.997010%
3,21,0.077358%,99.995549%
4,33,0.078067%,99.992942%


In [22]:
zcc = get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)
print(zcc)

In [23]:
dates_basedatos= Qcf.time_series_dates(fixings)
value_basedatos = Qcf.time_series_values(fixings)
x=len(value_basedatos)
ultima_fecha=dates_basedatos[x-1]
print(ultima_fecha)

28-10-2020


In [24]:
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois_cashflow: OisCashflow,
        zcc: Qcf.ZeroCouponCurve,
        fixings: Qcf.time_series) -> None:
    """
    Ver bien lo que hace la forward rate para poder sacar la tasa esperada. 
    Val_date: a la fecha que estoy calculando esa fecha esperada. El dia que estoy haciendo el calculo.
    Tengo un pedazo que es un factor de capitalizacion que ya conozco y un factor de capitalizacion que no
    conozco (de aqui al final del cupon), se deben juntar multiplicando. Fixings para el pasado y la curva 
    cero cupon para el futuro de los datos. Cuando estoy entre medio debo usar las dos. Fixing son la SOFR
    historicas, la curva del build sofr es la que calculamos a una cierta fecha (lo que esperamos que ocurra)
    
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Ver por ejemplo los casos cuando usamos el objeto Qcf.ForwardRates() en
    el notebook 9.
    """
    long = val_date.day_diff(ois_cashflow.end_date)
    short = val_date.day_diff(ois_cashflow.start_date)
    
    if short >= 0:
        short_rate = ois.get_accrued_rate(val_date, fixings)
        df_short = 1/(1+short_rate)**(short/360)
        
        long_rate = zcc.get_rate_at(abs(long))
        df_long = 1/(1+long_rate)**(long/360)
        
        expected_rate = (df_short / df_long  - 1) * 360.0 / (abs(long) - short)
        
    else:  
        short_rate = zcc.get_rate_at(short)
        df_short = 1/(1+short_rate)**(short/360)
        long_rate = zcc.get_rate_at(long)
        df_long = 1/(1+long_rate)**(long/360)

        expected_rate = (df_short / df_long  - 1) * 360.0 / (abs(long) - short)

    print(f"Expected Rate: {expected_rate: .6%}")
    ois_cashflow.interest_rate.set_value(expected_rate) 

    return

In [25]:
fecha=Qcf.QCDate(10, 10, 2019)
set_expected_rate(fecha, ois, zcc, fixings)

Expected Rate:  0.069869%


In [26]:
print(ois.interest_rate)

0.000699 Act360 Lin


In [27]:
print(ois.interest_rate.get_value())

0.000698691303813272


In [28]:
def present_value(
    val_date: Qcf.QCDate, 
    ois_cashflow: OisCashflow, 
    zcc: Qcf.ZeroCouponCurve) -> float:
    
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Para probar esta función utilizar la data de que está en data/20201012_built_sofr_zero.xlsx.
    Para traer a valor presente debes respetar la convención de las tasas de la curva.
    
    NOTA: la variable ois_cashflow debe haber 'pasado' por la función set_expected_rate para que
    tenga los intereses de los fixings futuros.
    """
    long = (val_date.day_diff(ois_cashflow.end_date))
    short = (val_date.day_diff(ois_cashflow.start_date))
    
    time_year = (long - short)/360
    
    notional=ois_cashflow.notional
    amort=ois_cashflow.amortization
    
    short_rate = zcc.get_rate_at(short)
    long_rate = zcc.get_rate_at(long)
    
    short_df=zcc.get_discount_factor_at(short)
    long_df=zcc.get_discount_factor_at(long)
    
    tasa=(short_df/long_df -1) *360 /(long- short)
    interest_rate= ois_cashflow.interest_rate.get_value()
    
    cash_flow = interest_rate *time_year * notional + amort 
    df = (tasa * time_year +1)**(-1)
    present_value = cash_flow * df
    
    return  present_value
    

In [29]:
fecha=Qcf.QCDate(10, 10, 2019)
present_value(fecha, ois, zcc)

10000104.445376925

## Tests

In [30]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [31]:
amount_tol = 10000
rate_tol = .001

In [32]:
exitos = 0

In [33]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

In [35]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

En la siguiente variable `exitos` se registra cuantos tests se superan con éxito.

### Objeto `fixings`

In [36]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [37]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [38]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [39]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

In [40]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued rate: {accrued_rate:.8%}')

accrued rate: 1.04766527%


Se verificará el cálculo usando `df_fixings`.

In [41]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [42]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [43]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [44]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [45]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [46]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [47]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

In [48]:
start_date = ois.start_date.description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [49]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.0075715527341267


In [50]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [51]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [52]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

La diferencia es: 0.03775213%


In [53]:
suma_exito(accrued_rate, check_accrued_rate, 'tasa')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.accrued_interest`

In [54]:
accrued_interest = ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued interest: {accrued_interest:,.2f}')

accrued interest: 75,082.68


In [55]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

La diferencia es: 0.000000


In [56]:
suma_exito(accrued_interest, check_accrued_interest, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.amount`

In [57]:
amount = ois.amount(fixings)
print(f'amount: {amount:,.2f}')

amount: 1,077,850.90


In [58]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [59]:
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2- 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {amount - check_amount:.6f}')

La diferencia es: 2823.854185


In [60]:
suma_exito(amount, check_amount, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [61]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [62]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [63]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [64]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

Fecha inicial flujo: 2019-10-01
Fecha final flujo: 2020-10-01


In [66]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Expected Rate:  0.873388%
Tasa esperada es: 0.87338758%
Check tasa esperada es: 0.06121277%
Suma medio punto por obtener resultado.


In [67]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Expected Rate:  0.070205%
Tasa esperada es: 0.07020480%
Check tasa esperada es: 0.06924275%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [68]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Expected Rate:  0.071717%
Tasa esperada es: 0.07171747%
Check tasa esperada es: 0.22166384%
Suma medio punto por obtener resultado.


In [69]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Expected Rate:  0.081112%
Tasa esperada es: 0.08111157%
Check tasa esperada es: 0.76574655%
Suma medio punto por obtener resultado.


In [70]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Expected Rate:  0.075347%
Tasa esperada es: 0.07534730%
Check tasa esperada es: 0.76574655%
Suma medio punto por obtener resultado.


### `present_value`

In [72]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Expected Rate:  0.873388%
Valor presente es: 1,088,117.2359
Amount: 1,077,850.8996
Df: 99.911652%
VP check es: 1,076,898.6424
Suma medio punto por obtener resultado.


In [73]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Expected Rate:  0.070205%
Valor presente es: 1,006,428.9941
Amount: 1,077,850.8996
Df: 99.929653%
VP check es: 1,077,092.6609
Suma medio punto por obtener resultado.


In [74]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Expected Rate:  0.071717%
Valor presente es: 1,006,567.4176
Amount: 1,077,850.8996
Df: 99.935019%
VP check es: 1,077,150.5062
Suma medio punto por obtener resultado.


In [75]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Expected Rate:  0.081112%
Valor presente es: 1,007,426.9689
Amount: 1,077,850.8996
Df: 0.000000%
VP check es: 0.0000
Suma medio punto por obtener resultado.


In [76]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

Expected Rate:  0.075347%
Valor presente es: 1,006,841.4107
Amount: 1,077,850.8996
Df: 0.000000%
VP check es: 0.0000
Suma medio punto por obtener resultado.


In [77]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 10.5
